# Web scraping a tienda en línea

Vamos a realizar un webscraping a alkosto, información pública

# Importaciones

In [153]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [154]:
driver = webdriver.Chrome()

In [155]:
url = 'https://www.alkosto.com/search?text=celulares'
driver.get(url)

In [157]:
time.sleep(5)

In [158]:
print("Página cargada, buscando productos...")
print(f"Productos iniciales: {len(driver.find_elements(By.CSS_SELECTOR, '#js-hits > div > ol > li'))}")

click_count = 0

while True:
    try:
        productos_actuales = len(driver.find_elements(By.CSS_SELECTOR, '#js-hits > div > ol > li'))
        print(f"\n--- Intento {click_count + 1} ---")
        print(f"Productos antes del clic: {productos_actuales}")
        
        # Esperar a que el botón esté presente
        boton_mas = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#js-hits > div > button'))
        )
        print("Botón encontrado")
        
        # Scroll al botón
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", boton_mas)
        time.sleep(1)
        
        # Click usando JavaScript (evita elementos que bloquean)
        print("Haciendo clic con JavaScript...")
        driver.execute_script("arguments[0].click();", boton_mas)
        click_count += 1
        
        # Esperar un momento para que cargue
        time.sleep(2)
        
        # Esperar a que aumenten los productos
        WebDriverWait(driver, 10).until(
            lambda d: len(d.find_elements(By.CSS_SELECTOR, '#js-hits > div > ol > li')) > productos_actuales
        )
        
        productos_nuevos = len(driver.find_elements(By.CSS_SELECTOR, '#js-hits > div > ol > li'))
        print(f"Productos después del clic: {productos_nuevos}")
        
    except TimeoutException:
        print("\nNo hay más productos para cargar o el botón desapareció")
        break
    except Exception as e:
        print(f"\nError: {type(e).__name__}: {e}")
        break

Página cargada, buscando productos...
Productos iniciales: 27

--- Intento 1 ---
Productos antes del clic: 27
Botón encontrado
Haciendo clic con JavaScript...
Productos después del clic: 52

--- Intento 2 ---
Productos antes del clic: 52
Botón encontrado
Haciendo clic con JavaScript...
Productos después del clic: 77

--- Intento 3 ---
Productos antes del clic: 77
Botón encontrado
Haciendo clic con JavaScript...
Productos después del clic: 102

--- Intento 4 ---
Productos antes del clic: 102
Botón encontrado
Haciendo clic con JavaScript...
Productos después del clic: 127

--- Intento 5 ---
Productos antes del clic: 127
Botón encontrado
Haciendo clic con JavaScript...
Productos después del clic: 152

--- Intento 6 ---
Productos antes del clic: 152
Botón encontrado
Haciendo clic con JavaScript...
Productos después del clic: 177

--- Intento 7 ---
Productos antes del clic: 177
Botón encontrado
Haciendo clic con JavaScript...
Productos después del clic: 202

--- Intento 8 ---
Productos ante

## Extracción

In [159]:
li_elements = driver.find_elements(By.CSS_SELECTOR,'#js-hits li')
data = []

In [160]:
for element in li_elements:
    try:
        titulo_element = element.find_element(By.TAG_NAME, 'h3')
        precio_element = element.find_element(By.CLASS_NAME, 'price')
        imagen_element = element.find_element(By.TAG_NAME, 'img').get_attribute('src')
    
        data.append({
            'Título': titulo_element.text,
            'Precio': precio_element.text,
            'Imagen': imagen_element
        })
    except NoSuchElementException:
        pass

In [161]:
data # al ejecutar esta línea de codigo se visualizan que los datos si queden bien guardados

[{'Título': 'Celular REDMI Note 14 Pro 256GB 4G Negro',
  'Precio': '$939.050',
  'Imagen': 'https://www.alkosto.com/medias/6941812742143-001-310Wx310H?context=bWFzdGVyfGltYWdlc3wzMzIyfGltYWdlL3dlYnB8YURGaEwyaGtaaTh4TkRnNU1EQXpOemczTURZeU1pODJPVFF4T0RFeU56UXlNVFF6WHpBd01WOHpNVEJYZURNeE1FZ3xhZjNlZGIxYjFkZDRiOWMxNTUwZWExODk4NTNmOGJiNGNlODFjOGVjYzc1YjRiMjM1ZDNiZTdhODY4YzEzM2Y3'},
 {'Título': 'Celular HONOR X8c 512GB 4G Verde',
  'Precio': '$949.050',
  'Imagen': 'https://www.alkosto.com/medias/6936520862337-001-310Wx310H?context=bWFzdGVyfGltYWdlc3w3MjQwfGltYWdlL3dlYnB8YURVMEwyZ3lOQzh4TlRBMk16azNNVEkyTmpVNU1DODJPVE0yTlRJd09EWXlNek0zWHpBd01WOHpNVEJYZURNeE1FZ3xkNTFhMTY3ZTA5MDY3YzM4YjllODhmNmQzNTgzYzVjYTQwNjk3NjcxZmVhM2E5MGE2Y2E0ODRhOTcwMjgzM2Fm'},
 {'Título': 'Celular Motorola Edge 60 Fusion 256GB 5G Café',
  'Precio': '$1.009.050',
  'Imagen': 'https://www.alkosto.com/medias/840493600086-001-310Wx310H?context=bWFzdGVyfGltYWdlc3w3MTkwfGltYWdlL3dlYnB8YUdFeUwyZzJNQzh4TlRNd01UTTNORGt3TWpNd01pOD

# Guardado de datos

In [162]:
df = pd.DataFrame(data)
df.to_csv('productos.csv')

In [163]:
driver.quit()